# Task 1 f, Calculating reserve factors against column buckling using Euler-Johnson

## We need from the data file:
- volume of the element
- Iyy calculated with `height_str, width_str, thickness_web, thickness_flange, thickness_skin, stringer_pitch`
- dimensions
- `EulerJohnson(EModulus, I_y, area, length, height_str, thickness_flange, thickness_web, radius, sigma_yield, sigma_applied, c=1)`

## Please enter your name (daniel, fabian, felix, yannis)

In [90]:
name = 'fabian'

## Imports

In [91]:
import pandas as pd
import numpy as np
import sys 
import os
sys.path.insert(0, os.path.abspath('..'))
sys.path.insert(0, os.path.abspath('../formulas'))

from formulas.columnbuckling import *
from formulas.panels import *
from formulas.strength import *
from formulas.helpers import *

## Constants

In [92]:
personal_data = personal_data_provider(name)
sigma_yield = personal_data[0]
EModulus = personal_data[1]
print(f"Your personal data is: sigma_yield = {sigma_yield}, EModulus = {EModulus}. Please verify.")

stringer_pitch = 200
effective_width = stringer_pitch/2

panel_element_length = 750/3
stringer_element_length = 750/3

# Hat stringer parameters
DIM1 = 25
DIM2 = 2
DIM3 = 20
DIM4 = 10

thickness_skin = 10


Your personal data is: sigma_yield = 490, EModulus = 65420.46. Please verify.


# Importing necessary files 

## Import everything for panels 

In [99]:
# Import panel properties and match the elements to the respective panel
paneldf = pd.read_csv(f'../data/{name}/panel_v2.csv')
element_component_df = pd.read_csv('../data/element_component_matching.csv')
paneldf = paneldf.rename(columns={'Elements':'Element ID'})
paneldf = pd.merge(paneldf, element_component_df, on='Element ID', how='left', suffixes=('_caller', '_other'))
paneldf.drop(columns=['FileID', 'Step', 'Layer'], inplace=True)
paneldf.head(5)

,Element ID,Loadcase,XX,XY,YY,YZ,ZX,ZZ,Component Name
0,1,1,-66.67,26.25,12.75,0.0,0.0,0.0,panel1
1,2,1,-82.97,27.10,20.87,0.0,0.0,0.0,panel1
2,3,1,-67.63,36.97,20.74,0.0,0.0,0.0,panel1
3,4,1,-97.67,34.52,2.42,0.0,0.0,0.0,panel2
4,5,1,-89.50,27.04,25.35,0.0,0.0,0.0,panel2


### We can already drop some columns from panel property

In [94]:
# Rearrange panel properties 
paneldf = paneldf.drop(columns=['YY','XY','YZ', 'ZX','ZZ'])
paneldf.head(2)

,Element ID,Loadcase,XX,Component Name
0,1,1,-66.67,panel1
1,2,1,-82.97,panel1


## Import everything for stringers 

In [95]:
# Open and match stringer properties 
stringerdf = pd.read_csv(f'../data/{name}/stringer_v2.csv', index_col=0)
stringerdf = stringerdf.rename(columns={'Element Stresses (1D):CBAR/CBEAM Axial Stress':'XX'})
stringerdf = stringerdf.drop(['FileID', 'Step'], axis = 1)
stringerdf = stringerdf.reset_index()
stringerdf = stringerdf.rename(columns={'Elements': 'Element ID'})
stringerdf = pd.merge(stringerdf, element_component_df, on='Element ID', how='left', suffixes=('_caller', '_other'))
stringerdf.tail(3)


,Element ID,Loadcase,XX,Component Name
78,64,3,-45.37,stringer9
79,65,3,-35.30,stringer9
80,66,3,-33.31,stringer9


# Now we rearrange the structure a bit

## First we split the 3 loadcases 

In [96]:
loadCase1dfPanel = paneldf[paneldf["Loadcase"] == 1]
loadCase2dfPanel = paneldf[paneldf["Loadcase"] == 2]
loadCase3dfPanel = paneldf[paneldf["Loadcase"] == 3]
loadCase1dfStringer = stringerdf[stringerdf["Loadcase"] == 1]
loadCase2dfStringer = stringerdf[stringerdf["Loadcase"] == 2]
loadCase3dfStringer = stringerdf[stringerdf["Loadcase"] == 3]
print(loadCase1dfPanel.head(5))
print(loadCase1dfStringer.head(5))

   Element ID  Loadcase     XX Component Name
0           1         1 -66.67         panel1
1           2         1 -82.97         panel1
2           3         1 -67.63         panel1
3           4         1 -97.67         panel2
4           5         1 -89.50         panel2
   Element ID  Loadcase     XX Component Name
0          40         1 -77.67      stringer1
1          41         1 -70.60      stringer1
2          42         1 -79.68      stringer1
3          43         1 -81.30      stringer2
4          44         1 -85.24      stringer2


# Now we need to combine the panels and the stringers

In [ ]:
# For every loadcase
# Efficiently combine panels for load case 1 
panel_groups_1 = []
for i in range(1, 10):
    df1 = loadCase1dfPanel[loadCase1dfPanel['Component Name'] == f'panel{i}'].copy()
    df2 = loadCase1dfPanel[loadCase1dfPanel['Component Name'] == f'panel{i+1}'].copy()
    df3 = loadCase1dfStringer[loadCase1dfStringer['Component Name']==f'stringer{i}'].copy()
    df1['Stiffener'] = 'stiffener'+str(i)
    df2['Stiffener'] = 'stiffener'+str(i)
    df3['Stiffener'] = 'stiffener'+str(i)
    panel_groups_1.extend([df1, df2, df3])
lc1combined = pd.concat(panel_groups_1, ignore_index=True)
lc1combined.head(10)

,Element ID,Loadcase,XX,Component Name,Stiffener
0,1,1,-66.67,panel1,stiffener1
1,2,1,-82.97,panel1,stiffener1
2,3,1,-67.63,panel1,stiffener1
3,4,1,-97.67,panel2,stiffener1
4,5,1,-89.50,panel2,stiffener1
5,6,1,-96.29,panel2,stiffener1
6,40,1,-77.67,stringer1,stiffener1
7,41,1,-70.60,stringer1,stiffener1
8,42,1,-79.68,stringer1,stiffener1
9,4,1,-97.67,panel2,stiffener2


In [104]:
# Efficiently combine panels for load case 2
panel_groups_2 = []
for i in range(1, 10):
    df1 = loadCase2dfPanel[loadCase2dfPanel['Component Name'] == f'panel{i}'].copy()
    df2 = loadCase2dfPanel[loadCase2dfPanel['Component Name'] == f'panel{i+1}'].copy()
    df3 = loadCase2dfStringer[loadCase2dfStringer['Component Name']==f'stringer{i}'].copy()
    df1['Stiffener'] = 'stiffener'+str(i)
    df2['Stiffener'] = 'stiffener'+str(i)
    df3['Stiffener'] = 'stiffener'+str(i)
    panel_groups_2.extend([df1, df2, df3])
lc2combined = pd.concat(panel_groups_2, ignore_index=True)
lc2combined.head(10)

,Element ID,Loadcase,XX,Component Name,Stiffener
0,1,2,-45.57,panel1,stiffener1
1,2,2,-55.38,panel1,stiffener1
2,3,2,-46.21,panel1,stiffener1
3,4,2,-64.34,panel2,stiffener1
4,5,2,-58.49,panel2,stiffener1
5,6,2,-63.42,panel2,stiffener1
6,40,2,-56.22,stringer1,stiffener1
7,41,2,-59.04,stringer1,stiffener1
8,42,2,-57.55,stringer1,stiffener1
9,4,2,-64.34,panel2,stiffener2


In [105]:
# Efficiently combine panels for load case 1 
panel_groups_3 = []
for i in range(1, 10):
    df1 = loadCase3dfPanel[loadCase3dfPanel['Component Name'] == f'panel{i}'].copy()
    df2 = loadCase3dfPanel[loadCase3dfPanel['Component Name'] == f'panel{i+1}'].copy()
    df3 = loadCase3dfStringer[loadCase3dfStringer['Component Name']==f'stringer{i}'].copy()
    df1['Stiffener'] = 'stiffener'+str(i)
    df2['Stiffener'] = 'stiffener'+str(i)
    df3['Stiffener'] = 'stiffener'+str(i)
    panel_groups_3.extend([df1, df2, df3])
lc3combined = pd.concat(panel_groups_3, ignore_index=True)
lc3combined.head(10)

,Element ID,Loadcase,XX,Component Name,Stiffener
0,1,3,-30.70,panel1,stiffener1
1,2,3,-41.48,panel1,stiffener1
2,3,3,-36.45,panel1,stiffener1
3,4,3,-52.64,panel2,stiffener1
4,5,3,-44.75,panel2,stiffener1
5,6,3,-44.34,panel2,stiffener1
6,40,3,-33.31,stringer1,stiffener1
7,41,3,-35.30,stringer1,stiffener1
8,42,3,-45.37,stringer1,stiffener1
9,4,3,-52.64,panel2,stiffener2


# Now we can aggregate the loadcases according to stiffeners

# Now we add Cross-Section Properties of the combined skin and hat stringer crosssection 

In [98]:
cs_properties = crosssectional_properties_hat_skin(DIM1, DIM2, DIM3, DIM4, thickness_skin, effective_width, stringer_depth=stringer_element_length)
effective_panel_volume = effective_width * thickness_skin * panel_element_length # 750 mm is the length of the panel, 3 FE per panel